<a href="https://colab.research.google.com/github/lakatosgabor/nav_uzemanyag_havi_info/blob/main/nav_uzemanyag_havi_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

url = "https://nav.gov.hu/print/ugyfeliranytu/uzemanyag/" + str(datetime.datetime.now().year) + "-ban-alkalmazhato-uzemanyagarak"

def get_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Dobjon kivételt, ha hiba történt a lekérdezésnél
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table")

        if table is None:
            raise ValueError("Nem található táblázat az oldalon.")
            return 0

        table_rows = table.find_all("tr")
        data = []
        for row in table_rows:
            cols = row.find_all(["th", "td"])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        if len(data) < 2:
            raise ValueError("A táblázatban nincsenek adatok.")
            return 0

        df = pd.DataFrame(data[1:], columns=data[0])

        if str(datetime.datetime.now().year) + '.év' not in df.columns or 'Gázolaj (Ft/l)' not in df.columns or 'LPG autógáz (Ft/l)' not in df.columns or 'Ólmozatlan motorbenzin ESZ-95\xa0 (Ft/l)' not in df.columns:
            raise ValueError("Hiányzó oszlopok a táblázatban.")
            return 0

        aktual_month = df[data[0][0]][0]
        disel = df[data[0][2]][0]
        lpg_gas = df[data[0][4]][0]
        petrol = df[data[0][1]][0]

        return aktual_month, disel, lpg_gas, petrol

    except requests.exceptions.HTTPError as errh:
        print("HTTP hiba történt:", errh)
        return 0
    except requests.exceptions.ConnectionError as errc:
        print("Kapcsolódási hiba történt:", errc)
        return 0
    except requests.exceptions.Timeout as errt:
        print("Időtúllépés történt:", errt)
        return 0
    except requests.exceptions.RequestException as err:
        print("Kérési hiba történt:", err)
        return 0
    except ValueError as ve:
        print(ve)
        return 0


def main():
  if (get_data(url) == 0):
    return

  aktual_month, disel, lpg_gas, petrol = get_data(url)

  if aktual_month is not None:
      query_time = datetime.datetime.now()
      print("Aktuális hónap üzemanyag árai: ")
      print("Hónap: ", aktual_month)
      print("Gázolaj: ", disel)
      print("LPG gáz: ", lpg_gas)
      print("Petrol: ", petrol)
      print("Lekérdezés ideje: ", query_time)
  else:
      print("Nem sikerült lekérdezni az adatokat.")


main()


Aktuális hónap üzemanyag árai: 
Hónap:  november
Gázolaj:  682
LPG gáz:  313
Petrol:  650
Lekérdezés ideje:  2023-10-27 09:16:58.208801
